In [ ]:
# from google.colab import drive
# drive.mount('/content/drive')

Mounted at /content/drive


In [1]:
import torch
from torch.utils.data import Dataset
import torch
import torch.nn as nn
import torch.optim as optim
import matplotlib.pyplot as plt
import pandas as pd
from datetime import datetime
import pandas as pd
import torch
from torch.utils.data import Dataset
import numpy as np
from torch.utils.data import DataLoader
import os

'''
human keypoints: 17*2
court keypoints: 6*2
net keypoints:   4*2
ball keypoints:  1*2
total keypoints: 28*2

'''

class HitDataset(Dataset):
    def __init__(self, dataset_folder, num_consecutive_frames, normalization=True):
        self.all_data = []  # Store all samples, regardless of being positive or negative
        self.dataset = []  # This will be populated dynamically
        self.normalization = normalization
        self.num_consecutive_frames = num_consecutive_frames
        self.balance = {"positive": 0, "negative": 0}  # Track the balance of samples

        # Load and preprocess data
        for root, dirs, files in os.walk(dataset_folder):
            for file in files:
                if file.endswith(".csv"):
                    data_path = os.path.join(root, file)
                    print(data_path)

                    try:
                        df = pd.read_csv(data_path, converters={"ball": eval, "top": eval, "bottom": eval, "court": eval, "net": eval})
                    except Exception as e:
                        print('Error! cannot process:', data_path, e)
                        continue

                    self.preprocess_df(df)

    def preprocess_df(self, df):
        rows = len(df)
        remainder = rows % self.num_consecutive_frames
        num_to_pad = (self.num_consecutive_frames - remainder) % self.num_consecutive_frames

        if num_to_pad > 0:
            last_row = df.iloc[-1]
            padding_data = np.tile(last_row.values, (num_to_pad, 1))
            padded_df = pd.DataFrame(padding_data, columns=df.columns)
            df = pd.concat([df, padded_df], axis=0).reset_index(drop=True)

        for i in range(0, len(df) - self.num_consecutive_frames + 1, self.num_consecutive_frames):
            oridata = df.loc[i:i + self.num_consecutive_frames - 1, :].copy().reset_index(drop=True)
            self.process_sequence(oridata)

    def process_sequence(self, sequence):
        data = []
        target1 = None

        for _, row in sequence.iterrows():
            pos = row['pos']
            if pos == 'top' and target1 is None:
                target1 = [0.0, 1.0, 0.0]
            elif pos == 'bottom' and target1 is None:
                target1 = [0.0, 0.0, 1.0]

            top = np.array(row['top']).reshape(-1, 2)
            bottom = np.array(row['bottom']).reshape(-1, 2)
            court = np.array(row['court']).reshape(-1, 2)
            ball = np.array(row['ball']).reshape(-1, 2)

            frame_data = np.concatenate((top, bottom, court, ball), axis=0)
            if self.normalization:
                frame_data[:, 0] /= 1920
                frame_data[:, 1] /= 1080
            data.append(frame_data.reshape(1, -1))

        data = np.array(data)
        if target1 is None:
            target1 = [1.0, 0.0, 0.0]  # Assume negative if not set

        self.all_data.append((data.reshape(-1), target1))

    def reset_epoch(self):
        self.dataset = []  # Clear previous epoch's data
        self.balance = {"positive": 0, "negative": 0}  # Reset balance

        for data, target in self.all_data:
            if target[1] == 1.0 or target[2] == 1.0:  # Positive sample
                self.dataset.append((data, target))
                self.balance["positive"] += 1
            elif self.balance["negative"] < self.balance["positive"]:  # Add negative sample to balance
                self.dataset.append((data, target))
                self.balance["negative"] += 1

    def __len__(self):
        return len(self.dataset)

    def __getitem__(self, index):
        sample = self.dataset[index]
        input_data, target1 = sample
        input_tensor = torch.tensor(input_data, dtype=torch.float32)
        target1_tensor = torch.tensor(target1, dtype=torch.float32)
        return input_tensor, target1_tensor

num_consecutive_frames=12
batch_size=30
shuffle=True
normalization=True

TrainDataset=HitDataset("./train",num_consecutive_frames,normalization)
ValidDataset=HitDataset("./valid",num_consecutive_frames,normalization)

./train/Ratchanok_INTANON_WANG_Zhi_Yi_Malaysia_Open_2022_SemiFinals/rally_1-24.csv
./train/Ratchanok_INTANON_WANG_Zhi_Yi_Malaysia_Open_2022_SemiFinals/rally_1-18.csv
./train/Ratchanok_INTANON_WANG_Zhi_Yi_Malaysia_Open_2022_SemiFinals/rally_1-25.csv
./train/Ratchanok_INTANON_WANG_Zhi_Yi_Malaysia_Open_2022_SemiFinals/rally_2-38.csv
./train/Ratchanok_INTANON_WANG_Zhi_Yi_Malaysia_Open_2022_SemiFinals/rally_2-39.csv
./train/Ratchanok_INTANON_WANG_Zhi_Yi_Malaysia_Open_2022_SemiFinals/rally_2-11.csv
./train/Ratchanok_INTANON_WANG_Zhi_Yi_Malaysia_Open_2022_SemiFinals/rally_2-9.csv
./train/Ratchanok_INTANON_WANG_Zhi_Yi_Malaysia_Open_2022_SemiFinals/rally_2-8.csv
./train/Ratchanok_INTANON_WANG_Zhi_Yi_Malaysia_Open_2022_SemiFinals/rally_2-14.csv
./train/Ratchanok_INTANON_WANG_Zhi_Yi_Malaysia_Open_2022_SemiFinals/rally_2-24.csv
./train/Ratchanok_INTANON_WANG_Zhi_Yi_Malaysia_Open_2022_SemiFinals/rally_2-26.csv
./train/Ratchanok_INTANON_WANG_Zhi_Yi_Malaysia_Open_2022_SemiFinals/rally_2-33.csv
./trai

KeyboardInterrupt: 

In [ ]:
class HitModel(nn.Module):
    def __init__(self, feature_dim, num_consecutive_frames):
        super(HitModel, self).__init__()
        self.num_consecutive_frames = num_consecutive_frames
        self.feature_dim = feature_dim

        self.gru1 = nn.GRU(feature_dim // num_consecutive_frames, 64, bidirectional=True, batch_first=True)
        self.gru2 = nn.GRU(128, 64, bidirectional=True, batch_first=True)
        self.global_maxpool = nn.MaxPool1d(num_consecutive_frames)
        self.dense = nn.Linear(128, 3)
        self.softmax = nn.Softmax(dim=-1)

    def forward(self, x):
        batch_size = x.shape[0]
        x=x.float()
        x = x.view(batch_size, self.num_consecutive_frames, self.feature_dim // self.num_consecutive_frames)
        x, _ = self.gru1(x)
        x, _ = self.gru2(x)
        x = x.transpose(1, 2)
        x = self.global_maxpool(x).squeeze()
        x= self.dense(x)
        x=self.softmax(x)
        return x

print(TrainDataset.positive,TrainDataset.negative)
print(ValidDataset.positive,ValidDataset.negative)

feature_dim=82*num_consecutive_frames
train_data_loader = DataLoader(TrainDataset, batch_size=batch_size, shuffle=shuffle)

valid_data_loader = DataLoader(ValidDataset, batch_size=batch_size, shuffle=shuffle)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model=HitModel(feature_dim,num_consecutive_frames)
criterion = nn.CrossEntropyLoss()
model.to(device)
criterion.to(device)
optimizer = optim.Adam(model.parameters(), lr=0.001)
num_epochs = 100



train_loss_list = []
valid_loss_list=[]
for epoch in range(num_epochs):
    train_loss_sum=0
    model.train()
    for batch_data in train_data_loader:
        inputs, labels = batch_data
        inputs = inputs.to(device)  # 将输入数据移动到设备上
        labels = labels.to(device)  # 将输入数据移动到设备上

        outputs = model(inputs)
        outputs=outputs.reshape(-1,3)

        train_loss = criterion(outputs,labels)
        train_loss_sum+=train_loss.detach()
        # 反向传播和优化
        optimizer.zero_grad()
        train_loss.backward()
        optimizer.step()
        # 执行你的训练或测试操作
    train_loss_list.append(train_loss_sum)


    # 打印训练信息
    if (epoch + 1) % 1== 0:
        print('Epoch [{}/{}], Loss: {:.4f}'.format(epoch + 1, num_epochs,
                                                    train_loss.item()))
    with torch.no_grad():
        correct = 0
        total = 0
        for inputs, labels in valid_data_loader:
            inputs = inputs.to(device)  # 将输入数据移动到设备上
            labels=labels.to(device)
            # 前向传播
            outputs = model(inputs)
            outputs=outputs.reshape(-1,3)


            y_true=torch.argmax(labels,axis=1)
            y_pred=torch.argmax(outputs,axis=1)
            total+=len(y_true)
            correct+=(y_true==y_pred).sum().item()
        if total==0:
            print(f'Accuracy on test set: {0}')
            continue



        print(f'Accuracy on test set: {correct/total}')

17846 17846
1152 1151
Epoch [1/100], Loss: 0.9240
Accuracy on test set: 0.6191923577941815
Epoch [2/100], Loss: 0.8772
Accuracy on test set: 0.7911419887103778
Epoch [3/100], Loss: 0.7710
Accuracy on test set: 0.7776812852800695
Epoch [4/100], Loss: 0.9756
Accuracy on test set: 0.8050369083803735
Epoch [5/100], Loss: 0.8513
Accuracy on test set: 0.7681285280069474
Epoch [6/100], Loss: 0.7698
Accuracy on test set: 0.803300043421624
Epoch [7/100], Loss: 1.0159
Accuracy on test set: 0.8219713417281806
Epoch [8/100], Loss: 0.8120
Accuracy on test set: 0.8345636126791142
Epoch [9/100], Loss: 0.8328
Accuracy on test set: 0.7993920972644377
Epoch [10/100], Loss: 0.8984
Accuracy on test set: 0.8467216673903604
Epoch [11/100], Loss: 0.7601
Accuracy on test set: 0.8241424229266174
Epoch [12/100], Loss: 0.7025
Accuracy on test set: 0.8393399913156752
Epoch [13/100], Loss: 0.7462
Accuracy on test set: 0.8310898827616153
Epoch [14/100], Loss: 0.8631
Accuracy on test set: 0.8376031263569258
Epoch [1

In [ ]:
# 保存整个模型
torch.save(model, '/content/drive/MyDrive/Badminton_Player_Analysis_Project/SoloShuttlePose/draft/HitDataset/hitdetect.pth')

# print(f"best_accuracy={best_accuracy}")
with torch.no_grad():
    correct = 0
    total = 0
    for inputs, labels in valid_data_loader:
        inputs = inputs.to(device)  # 将输入数据移动到设备上
        labels = labels.to(device)  # 将标签移动到设备上

        # 前向传播
        outputs = model(inputs)

        y_true=torch.argmax(labels,axis=1)
        y_pred=torch.argmax(outputs,axis=1)
        print(y_true)
        print(y_pred)
        total+=len(y_true)
        correct+=(y_true==y_pred).sum().item()


    print(f'Accuracy on test set: {correct/total}')




NameError: name 'valid_data_loader' is not defined

In [ ]:
final_list=[]
true_list=[]
data_path="/content/drive/MyDrive/Badminton_Player_Analysis_Project/SoloShuttlePose/draft/HitDataset/valid/Viktor_AXELSEN_Jonatan_CHRISTIE_Malaysia_Open_2022_SemiFinals/rally_1-12.csv"
df= pd.read_csv(data_path, converters={"ball": eval,"top":eval,"bottom":eval,"court":eval,"net":eval})

rows = len(df)
remainder = rows % 12
if remainder > 0:
    num_to_pad = 12 - remainder
else:
    num_to_pad = 0

if num_to_pad > 0:
    last_row = df.iloc[-1]
    padding_data = np.tile(last_row.values, (num_to_pad, 1))
    padded_df = pd.DataFrame(padding_data, columns=df.columns)
    df = pd.concat([df, padded_df], axis=0)
    df = df.reset_index(drop=True)

small_dataset =df
probs_list=[]
for i in range(len(small_dataset)):

    if i>=len(small_dataset)-num_consecutive_frames:
        break


    oridata=small_dataset.loc[i:i+num_consecutive_frames-1,:].copy()
    oridata=oridata.reset_index(drop=True)
    data=[]
    # print(oridata)
    if str(oridata.loc[0,'pos'])=='nan':
            true_list.append(0)
    elif str(oridata.loc[0,'pos'])=='top':
            true_list.append(1)
    elif str(oridata.loc[0,'pos'])=='bottom':
            true_list.append(2)

    for index,row in oridata.iterrows():

        top=np.array(row['top']).reshape(-1,2)
        bottom=np.array(row['bottom']).reshape(-1,2)
        court=np.array(row['court']).reshape(-1,2)
        # net=np.array(row['net']).reshape(-1,2)
        ball=np.array(row['ball']).reshape(-1,2)

        frame_data = np.concatenate((top, bottom, court, ball), axis=0)

        if normalization:
            # 将 x 坐标从 0 到 1920 映射到 1 到 2
            frame_data[:,0] /=1920

            # 将 y 坐标从 0 到 1080 映射到 1 到 2
            frame_data[:,1] /=1080

        data.append(frame_data.reshape(1,-1))
    data=np.array(data).reshape(1,-1)
    # print(data.shape)
    # if i%num_consecutive_frames!=0:
    #     final_list.append(0)
    #     continue
    outputs=model(torch.FloatTensor(data).to(device))

    # print(outputs)
    pred=torch.argmax(outputs).item()
    probs=outputs[pred].item()
    final_list.append(pred)

print(true_list)
print(final_list)


[2, 1, 2, 1, 2, 1, 2, 1, 2, 1, 2, 1, 2, 1, 2, 1, 2, 1, 2, 1]
[0, 0, 0, 0, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,